# Fun stuff

This notebook is a collection of a few **WTF** moments you could have as Python enthusiast.
Bottomline: This is not a recommended code style.

## HDL style signal assingment

To mimic VHDL or Verilog assignment style, you *could* be tempted to redefine your assignments to emulate:

```
a <= x - 1
```

This is simply done by inheriting from the `Signal` class:

In [1]:
from myirl import *
from myirl import targets
from myhdl import intbv

class HDLSignal(Signal):
    def __le__(self, other):
        return GenAssign(self, other)


/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: myirl.kernel.convertible.BinOp size changed, may indicate binary incompatibility. Expected 16 from C header, got 32 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: myirl.kernel.convertible.BoolOp size changed, may indicate binary incompatibility. Expected 16 from C header, got 32 from PyObject
  return f(*args, **kwds)


In [2]:
a, b = [ HDLSignal(intbv()[5:], name=n) for n in ['a', 'b'] ]

logic = kernel.sensitivity.LogicContext()
logic += [
    a <= 5,
    b <= a & b
]

In [3]:
d = DummyVHDLModule()
for stmt in logic:
    stmt.emit(d)

a <= "00101";
b <= a and b;


However, there's a catch: You will not be able to use the `<=` operator for instancing of a comparator.
With this abuse of the new `@` (matrix multiplication) operator, we can get some remedy (with side effects):

In [4]:
class MyOp(base.ConvertibleExpr):
    def __init__(self, func):
        self.func = func
    def __rmatmul__(self, other):
        return MyOp(lambda t, self=self, other=other : self.func(other, t))
    def __matmul__(self, other):
        return self.func(other)
    
ge = MyOp(lambda x, y: base.Ge(x, y)) # Generate an operator
le = MyOp(lambda x, y: base.Ge(y, x)) # swapped

In [5]:
a @le@ b

b >= a

In [6]:
expr = a @ge@ b

To verify it converts correctly, check:

In [7]:
expr, expr._convert(targets.VHDL, in_condition = False)

(a >= b, 'from_bool(a >= b)')